[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter5_garch_seminar_notebook.ipynb)

---

# Seminar: GARCH Models - Practice Exercises

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Exercises Overview

1. **Exercise 1:** ARCH-LM Testing and GARCH Estimation
2. **Exercise 2:** Model Comparison (GARCH vs GJR-GARCH vs EGARCH)
3. **Exercise 3:** Real Data Analysis - S&P 500
4. **Exercise 4:** Volatility Forecasting and VaR
5. **Exercise 5:** Bitcoin Volatility Analysis

## Setup

In [ ]:
# Install packages if needed (for Colab)
try:
    from arch import arch_model
    import yfinance as yf
except ImportError:
    !pip install arch yfinance --quiet
    from arch import arch_model
    import yfinance as yf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

from arch import arch_model
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22'}

print("Setup complete! Ready to analyze real financial data.")

---

## Exercise 1: ARCH-LM Testing and GARCH Estimation with Real Data

**Objective:** Learn to detect ARCH effects and estimate GARCH models using real market data.

**Tasks:**
1. Download EUR/USD exchange rate data
2. Apply the ARCH-LM test to detect heteroskedasticity
3. Estimate GARCH(1,1) model and interpret parameters

In [ ]:
# Task 1: Download real EUR/USD exchange rate data
print("Downloading EUR/USD exchange rate data from Yahoo Finance...")

eurusd = yf.download('EURUSD=X', start='2010-01-01', end='2024-12-31', progress=False)
eurusd_close = eurusd['Close'].squeeze() if isinstance(eurusd['Close'], pd.DataFrame) else eurusd['Close']

# Calculate returns (percentage)
returns = (eurusd_close.pct_change() * 100).dropna()
returns = pd.Series(returns.values, index=returns.index, name='returns')

print(f"\nEUR/USD Data Summary:")
print(f"Period: {returns.index[0].date()} to {returns.index[-1].date()}")
print(f"Observations: {len(returns)}")
print(f"\nBasic Statistics:")
print(f"  Mean return: {float(returns.mean()):.4f}%")
print(f"  Std deviation: {float(returns.std()):.4f}%")
print(f"  Skewness: {float(stats.skew(returns.values)):.4f}")
print(f"  Kurtosis: {float(stats.kurtosis(returns.values)+3):.4f} (Normal = 3)")

In [ ]:
# Task 2: Apply ARCH-LM test
print("ARCH-LM Test Results - EUR/USD")
print("="*50)

returns_demeaned = returns.values - returns.values.mean()
for lags in [5, 10, 20]:
    lm_stat, lm_pval, _, _ = het_arch(returns_demeaned, nlags=lags)
    result = "ARCH present" if lm_pval < 0.05 else "No ARCH"
    print(f"  Lags={lags:2d}: LM={lm_stat:8.2f}, p-value={lm_pval:.6f} → {result}")

# Visual check
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(returns.index, returns.values, color=COLORS['blue'], linewidth=0.5)
axes[0].set_title('EUR/USD Daily Returns (Volatility Clustering Visible)', fontweight='bold')
axes[0].set_ylabel('Return (%)')

plot_acf(returns.values**2, lags=30, ax=axes[1], color=COLORS['red'],
         vlines_kwargs={'color': COLORS['red']})
axes[1].set_title('ACF of Squared Returns (significant → ARCH effects)', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n→ ARCH effects confirmed in EUR/USD returns!")

In [ ]:
# Task 3: Estimate GARCH(1,1) on EUR/USD
model = arch_model(returns.values, vol='Garch', p=1, q=1, dist='t')
results = model.fit(disp='off')

print("GARCH(1,1) Estimation - EUR/USD")
print("="*50)
print(results.summary())

# Parameter interpretation
print("\nParameter Interpretation:")
omega = results.params['omega']
alpha = results.params['alpha[1]']
beta = results.params['beta[1]']
nu = results.params['nu']
persistence = alpha + beta

print(f"  ω = {omega:.6f}")
print(f"  α = {alpha:.4f} (news reaction)")
print(f"  β = {beta:.4f} (persistence)")
print(f"  α + β = {persistence:.4f} (total persistence)")
print(f"  ν = {nu:.2f} (Student-t degrees of freedom)")

if persistence < 1:
    uncond_vol = np.sqrt(omega / (1 - persistence))
    half_life = np.log(0.5) / np.log(persistence)
    print(f"\n  Unconditional volatility: {uncond_vol:.3f}% daily")
    print(f"  Half-life: {half_life:.1f} trading days")

---

## Exercise 2: Model Comparison with Real S&P 500 Data

**Objective:** Compare symmetric and asymmetric GARCH models on real equity data.

**Tasks:**
1. Download S&P 500 data (equity markets have strong leverage effect)
2. Estimate GARCH, GJR-GARCH, and EGARCH
3. Compare using AIC/BIC and test for leverage effect

In [ ]:
# Task 1: Download S&P 500 data
print("Downloading S&P 500 data from Yahoo Finance...")

sp500 = yf.download('^GSPC', start='2010-01-01', end='2024-12-31', progress=False)
sp500_close = sp500['Close'].squeeze() if isinstance(sp500['Close'], pd.DataFrame) else sp500['Close']

# Calculate returns
returns_sp = (sp500_close.pct_change() * 100).dropna()
returns_sp = pd.Series(returns_sp.values, index=returns_sp.index, name='returns')

print(f"\nS&P 500 Data Summary:")
print(f"Period: {returns_sp.index[0].date()} to {returns_sp.index[-1].date()}")
print(f"Observations: {len(returns_sp)}")
print(f"\nBasic Statistics:")
print(f"  Mean return: {float(returns_sp.mean()):.4f}%")
print(f"  Std deviation: {float(returns_sp.std()):.4f}%")
print(f"  Skewness: {float(stats.skew(returns_sp.values)):.4f} (negative → crashes)")
print(f"  Kurtosis: {float(stats.kurtosis(returns_sp.values)+3):.4f}")

# Check for leverage effect empirically
rolling_vol = returns_sp.rolling(window=5).std()
neg_returns = returns_sp.shift(1) < 0
vol_after_neg = rolling_vol[neg_returns].mean()
vol_after_pos = rolling_vol[~neg_returns].mean()
print(f"\nEmpirical Leverage Check:")
print(f"  Vol after negative days: {vol_after_neg:.3f}%")
print(f"  Vol after positive days: {vol_after_pos:.3f}%")
print(f"  Ratio: {vol_after_neg/vol_after_pos:.2f}x → Leverage effect present!")

In [ ]:
# Task 2: Estimate all three models on S&P 500
print("Model Estimation and Comparison - S&P 500")
print("="*60)

# GARCH(1,1)
model_garch = arch_model(returns_sp.values, vol='Garch', p=1, q=1, dist='t')
res_garch = model_garch.fit(disp='off')

# GJR-GARCH (o=1 adds asymmetry)
model_gjr = arch_model(returns_sp.values, vol='Garch', p=1, o=1, q=1, dist='t')
res_gjr = model_gjr.fit(disp='off')

# EGARCH (o=1 is needed for asymmetry term gamma)
model_egarch = arch_model(returns_sp.values, vol='EGARCH', p=1, o=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')

# Task 3: Compare
print(f"\n{'Model':<15} {'AIC':>10} {'BIC':>10} {'LogLik':>12}")
print("-"*50)
print(f"{'GARCH(1,1)':<15} {res_garch.aic:>10.2f} {res_garch.bic:>10.2f} {res_garch.loglikelihood:>12.2f}")
print(f"{'GJR-GARCH':<15} {res_gjr.aic:>10.2f} {res_gjr.bic:>10.2f} {res_gjr.loglikelihood:>12.2f}")
print(f"{'EGARCH':<15} {res_egarch.aic:>10.2f} {res_egarch.bic:>10.2f} {res_egarch.loglikelihood:>12.2f}")

# Identify best
models = {'GARCH': res_garch.aic, 'GJR-GARCH': res_gjr.aic, 'EGARCH': res_egarch.aic}
best = min(models, key=models.get)
print(f"\n→ Best model by AIC: {best}")
print("→ Asymmetric models outperform GARCH because S&P 500 has strong leverage effect!")

In [ ]:
# Check if leverage is significant in GJR-GARCH
print("\nGJR-GARCH: Leverage Effect Test - S&P 500")
print("="*50)

gamma_est = res_gjr.params['gamma[1]']
gamma_se = res_gjr.std_err['gamma[1]']
gamma_tstat = gamma_est / gamma_se
gamma_pval = 2 * (1 - stats.norm.cdf(abs(gamma_tstat)))

alpha_est = res_gjr.params['alpha[1]']

print(f"  α estimate: {alpha_est:.4f} (positive shock impact)")
print(f"  γ estimate: {gamma_est:.4f} (additional impact of negative shocks)")
print(f"  Std error:  {gamma_se:.4f}")
print(f"  t-stat:     {gamma_tstat:.2f}")
print(f"  p-value:    {gamma_pval:.6f}")
print(f"\n  Leverage significant? {'YES ✓' if gamma_pval < 0.05 else 'No'}")
print(f"\n  Negative shock impact: α + γ = {alpha_est + gamma_est:.4f}")
print(f"  Ratio: {(alpha_est + gamma_est)/alpha_est:.2f}x higher for negative shocks!")

---

## Exercise 3: Extended Analysis with Longer S&P 500 History

**Objective:** Apply GARCH modeling to long-term equity data covering multiple market regimes.

**Tasks:**
1. Download extended S&P 500 data (2000-2024) to include 2008 crisis
2. Calculate returns and test for ARCH effects
3. Estimate and diagnose GJR-GARCH model

In [ ]:
# Task 1: Get extended S&P 500 data (2000-2024)
print("Downloading extended S&P 500 data (2000-2024)...")

sp500_long = yf.download('^GSPC', start='2000-01-01', end='2024-12-31', progress=False)
sp500_long_close = sp500_long['Close'].squeeze() if isinstance(sp500_long['Close'], pd.DataFrame) else sp500_long['Close']

# Calculate returns
returns_long = (sp500_long_close.pct_change() * 100).dropna()
returns_long = pd.Series(returns_long.values, index=returns_long.index, name='returns')

print(f"\nExtended S&P 500 Data:")
print(f"Period: {returns_long.index[0].date()} to {returns_long.index[-1].date()}")
print(f"Observations: {len(returns_long)} (covers 2008 crisis, COVID, etc.)")
print(f"\nDescriptive Statistics:")
print(returns_long.describe())

In [ ]:
# Visualize extended S&P 500 data with crisis periods
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(returns_long.index, returns_long.values, color=COLORS['blue'], linewidth=0.5)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_ylabel('Return (%)')
axes[0].set_title('S&P 500 Daily Returns (2000-2024)', fontweight='bold')

# Mark crisis periods
crisis_periods = [
    ('2008-09-01', '2009-03-31', '2008 Crisis', COLORS['red']),
    ('2020-02-15', '2020-04-30', 'COVID-19', COLORS['orange']),
    ('2022-01-01', '2022-10-31', '2022 Bear', COLORS['green'])
]
for start, end, label, color in crisis_periods:
    axes[0].axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.2, color=color, label=label)
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)

# Rolling 20-day volatility
rolling_vol = returns_long.rolling(window=20).std()
axes[1].plot(rolling_vol.index, rolling_vol.values, color=COLORS['red'], linewidth=0.8)
axes[1].set_ylabel('Rolling Volatility (%)')
axes[1].set_xlabel('Date')
axes[1].set_title('20-Day Rolling Volatility', fontweight='bold')

for start, end, label, color in crisis_periods:
    axes[1].axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.2, color=color)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  • 2008 financial crisis: Extreme volatility spike (~10% daily moves)")
print("  • COVID-19 crash: Sharp but short volatility spike")
print("  • 2022 bear market: Elevated but moderate volatility")

In [ ]:
# Task 2: Test for ARCH effects
print("ARCH-LM Test on Extended S&P 500 Data")
print("="*50)

returns_demeaned = returns_long.values - returns_long.values.mean()
for lags in [5, 10, 20]:
    lm_stat, lm_pval, _, _ = het_arch(returns_demeaned, nlags=lags)
    print(f"  Lags={lags:2d}: LM={lm_stat:8.2f}, p-value={lm_pval:.6f}")

print("\n→ Very strong ARCH effects present in long-term S&P 500 data!")

In [ ]:
# Task 3: Estimate GJR-GARCH with Student-t on extended data
model_long = arch_model(returns_long.values, vol='Garch', p=1, o=1, q=1, dist='t')
res_long = model_long.fit(disp='off')

print("GJR-GARCH(1,1,1) with Student-t - Extended S&P 500 (2000-2024)")
print("="*60)
print(res_long.summary())

In [ ]:
# Interpretation of extended model
print("\nParameter Interpretation - Extended S&P 500")
print("="*50)

alpha_long = res_long.params['alpha[1]']
gamma_long = res_long.params['gamma[1]']
beta_long = res_long.params['beta[1]']
nu_long = res_long.params['nu']

persistence = alpha_long + gamma_long/2 + beta_long
half_life = np.log(0.5) / np.log(persistence) if persistence < 1 else float('inf')

print(f"  α = {alpha_long:.4f} (news reaction)")
print(f"  γ = {gamma_long:.4f} (leverage effect)")
print(f"  β = {beta_long:.4f} (persistence)")
print(f"  ν = {nu_long:.2f} (Student-t degrees of freedom)")
print(f"\n  Persistence (α + γ/2 + β) = {persistence:.4f}")
print(f"  Half-life = {half_life:.1f} trading days")
print(f"\n  Leverage ratio: {(alpha_long + gamma_long)/alpha_long:.2f}x higher impact for negative shocks")

# Plot conditional volatility
fig, ax = plt.subplots(figsize=(14, 5))
cond_vol = res_long.conditional_volatility
ax.plot(returns_long.index, cond_vol, color=COLORS['red'], linewidth=0.8)
ax.fill_between(returns_long.index, 0, cond_vol, color=COLORS['red'], alpha=0.3)
ax.set_ylabel('Conditional Volatility (%)')
ax.set_xlabel('Date')
ax.set_title('GJR-GARCH Conditional Volatility - S&P 500 (2000-2024)', fontweight='bold')

for start, end, label, color in crisis_periods:
    ax.axvspan(pd.Timestamp(start), pd.Timestamp(end), alpha=0.15, color='gray')

plt.tight_layout()
plt.show()

In [ ]:
# Diagnostics on extended model
std_resid_long = res_long.std_resid

print("Diagnostic Tests - Extended S&P 500")
print("="*50)

# Ljung-Box on squared residuals
lb_test = acorr_ljungbox(std_resid_long**2, lags=10, return_df=True)
print(f"Ljung-Box (z²): Q(10) = {lb_test['lb_stat'].iloc[-1]:.2f}, p-value = {lb_test['lb_pvalue'].iloc[-1]:.4f}")

# ARCH-LM on standardized residuals
lm_stat, lm_pval, _, _ = het_arch(std_resid_long, nlags=5)
print(f"ARCH-LM (5 lags): LM = {lm_stat:.2f}, p-value = {lm_pval:.4f}")

if lm_pval > 0.05:
    print("\n✓ No remaining ARCH effects - GJR-GARCH model is adequate!")
else:
    print("\n⚠️ Some ARCH effects remain - consider alternative specifications")

---

## Exercise 4: Volatility Forecasting and VaR

**Objective:** Generate volatility forecasts and calculate Value at Risk.

**Tasks:**
1. Forecast volatility 10 days ahead
2. Calculate 1-day and 10-day VaR
3. Backtest VaR

In [ ]:
# Task 1: Volatility forecast using extended S&P 500 model
horizon = 10
forecasts = res_long.forecast(horizon=horizon)
vol_forecast = np.sqrt(forecasts.variance.values[-1, :])

print("Volatility Forecast (next 10 days) - S&P 500")
print("="*40)
for h in range(horizon):
    print(f"  Day {h+1}: {vol_forecast[h]:.3f}%")

# Plot
fig, ax = plt.subplots(figsize=(12, 5))

# Last 50 days historical
hist_vol = res_long.conditional_volatility
ax.plot(range(50), hist_vol[-50:], color=COLORS['blue'], label='Historical')
# Use [-1] instead of .iloc[-1] since it might be numpy array
last_hist_vol = hist_vol[-1] if isinstance(hist_vol, np.ndarray) else hist_vol.iloc[-1]
ax.plot(range(49, 49+horizon+1), np.concatenate([[last_hist_vol], vol_forecast]),
        color=COLORS['red'], linestyle='--', linewidth=2, label='Forecast')
ax.axvline(x=49, color='black', linestyle='-', alpha=0.3)
ax.set_xlabel('Days')
ax.set_ylabel('Volatility (%)')
ax.set_title('S&P 500 Volatility Forecast', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

plt.tight_layout()
plt.show()

In [ ]:
# Task 2: Calculate VaR
portfolio_value = 1_000_000  # EUR
sigma_1day = vol_forecast[0] / 100

# Quantiles
nu = res_long.params['nu']
q_95_norm = stats.norm.ppf(0.95)
q_99_norm = stats.norm.ppf(0.99)
q_95_t = stats.t.ppf(0.95, df=nu) * np.sqrt((nu-2)/nu)
q_99_t = stats.t.ppf(0.99, df=nu) * np.sqrt((nu-2)/nu)

print("Value at Risk Calculation - S&P 500 Portfolio")
print("="*50)
print(f"Portfolio: €{portfolio_value:,.0f}")
print(f"1-day volatility: {sigma_1day*100:.3f}%")

print("\n1-Day VaR:")
print(f"  Normal 95%: €{q_95_norm * sigma_1day * portfolio_value:,.0f}")
print(f"  Normal 99%: €{q_99_norm * sigma_1day * portfolio_value:,.0f}")
print(f"  Student-t 95%: €{q_95_t * sigma_1day * portfolio_value:,.0f}")
print(f"  Student-t 99%: €{q_99_t * sigma_1day * portfolio_value:,.0f}")

# 10-day VaR (sqrt(10) scaling)
print("\n10-Day VaR (scaled):")
print(f"  Normal 99%: €{q_99_norm * sigma_1day * portfolio_value * np.sqrt(10):,.0f}")
print(f"  Student-t 99%: €{q_99_t * sigma_1day * portfolio_value * np.sqrt(10):,.0f}")

print(f"\n→ Student-t VaR is {(q_99_t/q_99_norm - 1)*100:.1f}% higher than Normal VaR")

In [ ]:
# Task 3: Simple VaR backtest on S&P 500
print("\nVaR Backtest (95% level) - S&P 500")
print("="*50)

# Calculate 1-day VaR for each day
cond_vol = res_long.conditional_volatility / 100
VaR_95 = q_95_t * cond_vol

# Count violations (returns < -VaR)
returns_dec = returns_long / 100
violations = returns_dec < -VaR_95
violation_rate = float(violations.mean())
expected_rate = 0.05

print(f"Expected violation rate: {expected_rate*100:.1f}%")
print(f"Actual violation rate: {violation_rate*100:.2f}%")
print(f"Number of violations: {int(violations.sum())} out of {len(violations)}")

# Kupiec test (proportion of failures)
n_total = len(violations)
n_viol = int(violations.sum())

# Handle edge case
if violation_rate > 0 and violation_rate < 1:
    LR = 2 * (n_viol * np.log(violation_rate/expected_rate) + 
              (n_total - n_viol) * np.log((1-violation_rate)/(1-expected_rate)))
    pval = 1 - stats.chi2.cdf(LR, 1)
    print(f"\nKupiec LR test: LR = {LR:.2f}, p-value = {pval:.4f}")
    print(f"Conclusion: {'VaR model adequate ✓' if pval > 0.05 else 'VaR model needs adjustment'}")
else:
    print("\nKupiec test not applicable (extreme violation rate)")

---

## Exercise 5: Bitcoin Volatility Analysis

**Objective:** Analyze cryptocurrency volatility and compare with traditional assets.

**Tasks:**
1. Download Bitcoin data and calculate returns
2. Estimate GARCH models
3. Compare volatility characteristics with S&P 500

In [ ]:
# Task 1: Get Bitcoin data from Yahoo Finance
print("Downloading Bitcoin data from Yahoo Finance...")

btc = yf.download('BTC-USD', start='2017-01-01', end='2024-12-31', progress=False)
btc_close = btc['Close'].squeeze() if isinstance(btc['Close'], pd.DataFrame) else btc['Close']

# Calculate returns
returns_btc = (btc_close.pct_change() * 100).dropna()
returns_btc = pd.Series(returns_btc.values, index=returns_btc.index, name='returns')

print(f"\nBitcoin Data Summary:")
print(f"Period: {returns_btc.index[0].date()} to {returns_btc.index[-1].date()}")
print(f"Observations: {len(returns_btc)}")
print(f"\nBasic Statistics:")
print(f"  Mean return: {float(returns_btc.mean()):.4f}%")
print(f"  Std deviation: {float(returns_btc.std()):.4f}%")
print(f"  Min: {float(returns_btc.min()):.2f}%")
print(f"  Max: {float(returns_btc.max()):.2f}%")
print(f"  Skewness: {float(stats.skew(returns_btc.values)):.4f}")
print(f"  Kurtosis: {float(stats.kurtosis(returns_btc.values)+3):.4f}")

In [ ]:
# Visualize Bitcoin returns
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(returns_btc.index, returns_btc.values, color=COLORS['orange'], linewidth=0.5)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_ylabel('Return (%)')
axes[0].set_title('Bitcoin Daily Returns (2017-2024)', fontweight='bold')

axes[1].hist(returns_btc.values, bins=100, density=True, color=COLORS['orange'],
             alpha=0.7, edgecolor='white')
x = np.linspace(-30, 30, 100)
axes[1].plot(x, stats.norm.pdf(x, float(returns_btc.mean()), float(returns_btc.std())),
             color=COLORS['red'], linewidth=2, label='Normal')
kurtosis_btc = float(stats.kurtosis(returns_btc.values) + 3)
axes[1].set_title(f'Distribution (Kurtosis = {kurtosis_btc:.1f})', fontweight='bold')
axes[1].set_xlim(-30, 30)
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nKey observation: Bitcoin has much higher volatility and fatter tails than S&P 500!")

In [ ]:
# Task 2: Estimate GJR-GARCH model for Bitcoin
model_btc = arch_model(returns_btc.values, vol='Garch', p=1, o=1, q=1, dist='t')
res_btc = model_btc.fit(disp='off')

print("GJR-GARCH(1,1,1) Estimation - Bitcoin")
print("="*50)
print(res_btc.summary())

In [ ]:
# Task 3: Comparison with S&P 500
print("\nComparison: Bitcoin vs S&P 500 (Real Data)")
print("="*60)

# Use comparable period
common_start = max(returns_long.index[0], returns_btc.index[0])
common_end = min(returns_long.index[-1], returns_btc.index[-1])
sp_comp = returns_long[(returns_long.index >= common_start) & (returns_long.index <= common_end)]
btc_comp = returns_btc[(returns_btc.index >= common_start) & (returns_btc.index <= common_end)]

print(f"\nComparable period: {common_start.date()} to {common_end.date()}")
print(f"\n{'Metric':<25} {'S&P 500':>12} {'Bitcoin':>12}")
print("-"*50)
print(f"{'Mean return (%)':<25} {float(sp_comp.mean()):>12.3f} {float(btc_comp.mean()):>12.3f}")
print(f"{'Volatility (%)':<25} {float(sp_comp.std()):>12.3f} {float(btc_comp.std()):>12.3f}")
print(f"{'Skewness':<25} {float(stats.skew(sp_comp.values)):>12.3f} {float(stats.skew(btc_comp.values)):>12.3f}")
print(f"{'Kurtosis':<25} {float(stats.kurtosis(sp_comp.values)+3):>12.3f} {float(stats.kurtosis(btc_comp.values)+3):>12.3f}")

# Model parameters comparison
print(f"\n{'GJR-GARCH Parameters:':<25}")
print(f"{'α (news reaction)':<25} {res_long.params['alpha[1]']:>12.4f} {res_btc.params['alpha[1]']:>12.4f}")
print(f"{'γ (leverage)':<25} {res_long.params['gamma[1]']:>12.4f} {res_btc.params['gamma[1]']:>12.4f}")
print(f"{'β (persistence)':<25} {res_long.params['beta[1]']:>12.4f} {res_btc.params['beta[1]']:>12.4f}")
print(f"{'ν (Student-t df)':<25} {res_long.params['nu']:>12.2f} {res_btc.params['nu']:>12.2f}")

# Calculate persistence
pers_sp = res_long.params['alpha[1]'] + res_long.params['gamma[1]']/2 + res_long.params['beta[1]']
pers_btc = res_btc.params['alpha[1]'] + res_btc.params['gamma[1]']/2 + res_btc.params['beta[1]']
print(f"{'Persistence (α+γ/2+β)':<25} {pers_sp:>12.4f} {pers_btc:>12.4f}")

vol_ratio = float(btc_comp.std()) / float(sp_comp.std())
print(f"\nKey findings:")
print(f"  • Bitcoin volatility is ~{vol_ratio:.1f}x higher than S&P 500")
print(f"  • Bitcoin has {'higher' if res_btc.params['alpha[1]'] > res_long.params['alpha[1]'] else 'lower'} α (faster reaction to news)")
print(f"  • Bitcoin has {'weaker' if res_btc.params['gamma[1]'] < res_long.params['gamma[1]'] else 'stronger'} leverage effect")

---

## Summary

### What We Learned (Using Real Financial Data)

1. **ARCH-LM test** reliably detects heteroskedasticity in real market data
   - S&P 500, EUR/USD, and Bitcoin all show strong ARCH effects

2. **GARCH(1,1)** captures volatility dynamics well
   - α measures news reaction (higher for Bitcoin than S&P 500)
   - β measures persistence (high for all assets, ~0.85-0.95)
   - α + β < 1 for stationarity

3. **Asymmetric models** (GJR, EGARCH) capture leverage effect
   - S&P 500 shows strong leverage effect (γ significant)
   - Bitcoin shows weaker leverage effect
   - Negative returns increase volatility more than positive returns

4. **Student-t distribution** essential for real financial data
   - Degrees of freedom typically 5-10
   - Produces higher VaR estimates than Normal

5. **VaR calculation** with GARCH is practical
   - 1-day VaR: ~2-3% of portfolio at 99% confidence
   - Scale by √T for multi-day horizons

6. **Bitcoin vs S&P 500**
   - Bitcoin ~3-4x more volatile
   - Bitcoin reacts faster to news (higher α)
   - S&P 500 has stronger leverage effect

### Practical Workflow for Real Data
1. Download data (Yahoo Finance via yfinance)
2. Calculate returns and basic statistics
3. Test for ARCH effects (ARCH-LM test)
4. Estimate GJR-GARCH with Student-t
5. Check diagnostics (Ljung-Box, ARCH-LM on residuals)
6. Forecast volatility and calculate VaR